# <center> POS-Tagging dengan Metode Baseline </center>

### <center> A. Pembangunan Model Data Train <center>

In [21]:
def read_file(fname):
    with open(fname) as f:
        content = f.readlines()
    content = [x.strip() for x in content]
    return content

In [22]:
def frekuensi(content):
    tag_count = {} #menyimpan jumlah kemunculan tag
    tag_count['<start>'] = 0 
    word_tag = {} #jumlah sebuah kata diberi tag tertentu
    idx_line = 0
    is_first_word = 0
    
    while idx_line < len(content):
        prev_tag = '<start>'
        while not content[idx_line].startswith('</kalimat'):
            #Jika bukan kata pertama dalam kalimat
            if  not content[idx_line].startswith('<kalimat'):
                content_part = content[idx_line].split('\t')
                if content_part[1] in tag_count:
                    tag_count[content_part[1]] += 1
                else:
                    tag_count[content_part[1]] = 1
                    
                if (content_part[0],content_part[1]) in word_tag: #keys dibiarkan menjadi terpisah 
                    word_tag[(content_part[0],content_part[1])] += 1 #untuk memudahkan ketika mengambil indeks keys dictionary
                else:    
                    word_tag[(content_part[0],content_part[1])] = 1 #dan juga agar tanda ',' yg memisahkan kata dengan tag tidak dibaca sebagai string
                  
        #Jika kata pertama dalam kalimat    
            else:
                tag_count['<start>'] += 1
                is_first_word = 1
            idx_line = idx_line + 1

        idx_line = idx_line+1       
    return tag_count, word_tag

In [23]:
jumlah_tag ={}
jumlah_emission = {}
jumlah_transisi = {}
content = read_file('Data_Train.txt')
jumlah_tag, jumlah_emission = frekuensi(content) #membangun dictionary untuk jumlah kemunculan tag dan jumlah kemunculan kata diberi tag tertentu

## <center> B. Pengujian Model Data Train</center>

In [24]:
test = read_file('Data_Test.txt')
sentences= []
tags = []
index = 0
while index < len(test):
    sent = [] #untuk menampung katanya
    tag = [] #untuk menampung postag dari tiap kata
    while not test[index].startswith('</kalimat'): #batas akhir dari kalimat
        if  not test[index].startswith('<kalimat'):
            content_part = test[index].split('\t')
            sent.append(content_part[0])
            tag.append(content_part[1])
        index = index + 1
    sentences.append(sent)
    tags.append(tag)
    index = index+2   

In [25]:
kumpulan_kata = [] 
kumpulan_tag = []
                                       #Mengambil setiap kata dalam kalimat dalam data test dan dijadikan dalam satu array yang menampung seluruh kata
for sentence in sentences:
    for i in range(len(sentence)):
        kumpulan_kata.append(sentence[i]) 
        
                                       #Mengambil setiap tag dari seluruh kata dalam data test dan dijadikan dalam satu array yang menampung seluruh tag
for tag in tags:
    for i in range(len(tag)):
        kumpulan_tag.append(tag[i])

In [26]:
keys = jumlah_emission.keys()
word_keys=[]
for word in keys:
    word_keys.append(word[0]) #untuk mengambil kata yang menjadi keys didalam dictionary jumlah kemunculan kata diberi tag tertentu

In [27]:
word_tag=[] #array yang akan menampung seluruh kemungkinan tag terhadap kata yang ada didalam data test
for kata in kumpulan_kata:
    if kata in word_keys: #mengecek apakah kata dalam data test sudah pernah terbaca didalam model
        for part_keys in keys:
            if kata == part_keys[0]: #jika sudah pernah dibaca maka dicocokkan kata tersebut dengan kata yg ada didalam dictionary dan mengambil pasangan tag dari kata tersebut dan juga jumlah kemunculannya
                word_tag.append((kata,part_keys[1],jumlah_emission[(kata,part_keys[1])]))
    else:
        jumlah_emission[(kata,'NN')] = 1 #jika belum pernah dibaca maka akan dimasukkan kedalam dictionary jumlah kemunculan kata tersebut dan diberi tag NN
        word_tag.append((kata,'NN',1))
print(word_tag)

[('Perusahaan', 'NN', 20), ('Perusahaan', 'NNP', 13), ('ritel', 'NN', 1), ('PT', 'NNP', 159), ('Matahari', 'NNP', 1), ('Putra', 'NNP', 1), ('Prima', 'NNP', 3), ('Tbk', 'NNP', 51), ('MPPA', 'NN', 1), ('dalam', 'IN', 174), ('dalam', 'NN', 6), ('dalam', 'SC', 6), ('dalam', 'X', 1), ('tiga', 'CD', 18), ('bulan', 'NN', 44), ('pertama', 'OD', 39), ('yang', 'SC', 538), ('yang', 'DT', 1), ('berakhir', 'VB', 5), ('Maret', 'NNP', 14), ('2007', 'CD', 164), ('2007', 'NNP', 5), ('berhasil', 'VB', 11), ('meraih', 'VB', 9), ('laba bersih', 'NN', 23), ('Rp', 'SYM', 267), ('15,18', 'NN', 1), ('miliar', 'CD', 120), (',', 'Z', 1425), ('atau', 'CC', 96), ('naik', 'VB', 38), ('24,43', 'NN', 1), ('persen', 'CD', 369), ('dari', 'IN', 251), ('Rp', 'SYM', 267), ('12,20', 'NN', 1), ('miliar', 'CD', 120), ('pada', 'IN', 272), ('periode', 'NN', 46), ('sama', 'JJ', 38), ('tahun', 'NN', 168), ('tahun', 'NNP', 3), ('lalu', 'NN', 1), ('lalu', 'CC', 35), ('.', 'Z', 1012), ('.', 'NNP', 14), ('Penjualan', 'NN', 3), ('be

In [28]:
#Melakukan eliminasi terhadap kata yang memiliki lebih dari satu tag yang berbeda berdasarkan jumlah kemunculan terbanyak
fix_word_tag = word_tag.copy()
for i in range(len(word_tag)-1):
    if word_tag[i] in fix_word_tag:
        if word_tag[i][0] == word_tag[i+1][0] and word_tag[i][1] != word_tag[i+1]: #eliminasi hanya berlaku apabila ada tag yang berbeda dengan kata yang sama
            if word_tag[i][2] > word_tag[i+1][2]:
                fix_word_tag.remove(word_tag[i+1])
            elif word_tag[i][2] < word_tag[i+1][2] and word_tag[i] in fix_word_tag: #apabila kata yang mau dieliminasi masih terdapat didalam array
                fix_word_tag.remove(word_tag[i])
            elif word_tag[i][2] == word_tag[i+1][2]:
                fix_word_tag.remove(word_tag[i+1])

In [29]:
#Melakukan eliminasi lagi seperti diatas, karena masih ada kata yang sama tetapi memiliki lebih dari satu tag
fix_fix = fix_word_tag.copy()
for i in range(len(fix_word_tag)-1):
    if fix_word_tag[i] in fix_fix:
        if fix_word_tag[i][0] == fix_word_tag[i+1][0] and word_tag[i][1] != word_tag[i+1]: #eliminasi hanya berlaku apabila ada tag yang berbeda dengan kata yang sama
            if fix_word_tag[i][2] > fix_word_tag[i+1][2]:
                fix_fix.remove(fix_word_tag[i+1])
            elif fix_word_tag[i][2] < fix_word_tag[i+1][2]:
                fix_fix.remove(fix_word_tag[i])
            elif fix_word_tag[i][2] == fix_word_tag[i+1][2]:
                fix_fix.remove(fix_word_tag[i+1])
# print(fix_fix)

In [30]:
tag_test = [] #menampung hasil tag dari proses pengujian model
for i in range(len(fix_fix)):
    tag_test.append(fix_fix[i][1]) #mengambil seluruh tag yang ada didalam array hasil akhir

In [31]:
# Melakukan perhitungan akurasi
counter=0
jumlah_salah = 0
for i in range(len(tag_test)):
    if tag_test[i] == kumpulan_tag[i]:
        counter += 1
    elif tag_test[i] != kumpulan_tag[i]:
        jumlah_salah +=1
        print(jumlah_salah , 'Kata: '+ fix_fix[i][0], 'Tag Test: '+tag_test[i], 'Tag Asli: '+kumpulan_tag[i])
#         print('---------------------')
print(jumlah_salah)
akurasi = (counter/len(kumpulan_tag)) * 100
print('Akurasi Data Test: ',akurasi)

1 Kata: MPPA Tag Test: NN Tag Asli: NNP
2 Kata: 15,18 Tag Test: NN Tag Asli: CD
3 Kata: 24,43 Tag Test: NN Tag Asli: CD
4 Kata: 12,20 Tag Test: NN Tag Asli: CD
5 Kata: 1,68 Tag Test: NN Tag Asli: CD
6 Kata: 2,02 Tag Test: NN Tag Asli: CD
7 Kata: menyusut Tag Test: NN Tag Asli: VB
8 Kata: 58,27 Tag Test: NN Tag Asli: CD
9 Kata: 41,39 Tag Test: NN Tag Asli: CD
10 Kata: memangkas Tag Test: NN Tag Asli: VB
11 Kata: 41,98 Tag Test: NN Tag Asli: CD
12 Kata: 34,11 Tag Test: NN Tag Asli: CD
13 Kata: mencetak Tag Test: NN Tag Asli: VB
14 Kata: 160,50 Tag Test: NN Tag Asli: CD
15 Kata: 30,5 Tag Test: NN Tag Asli: CD
16 Kata: diantaranya Tag Test: NN Tag Asli: X
17 Kata: dibagikan Tag Test: NN Tag Asli: VB
18 Kata: 49 Tag Test: NN Tag Asli: CD
19 Kata: - Tag Test: Z Tag Asli: NNP
20 Kata: 8 Tag Test: CD Tag Asli: NNP
21 Kata: 16,9 Tag Test: NN Tag Asli: CD
22 Kata: sekitar Tag Test: RB Tag Asli: IN
23 Kata: 14,18 Tag Test: NN Tag Asli: CD
24 Kata: 14,8 Tag Test: NN Tag Asli: CD
25 Kata: Hidajat T